In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
#import gdown
from zipfile import ZipFile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation : Flipping

# Data Augmentation : Noise

# Train / Test dataset 만들기

In [2]:
#colab에 googld drive 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset = keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/의학영상처리/dataset2", labels="inferred", label_mode="binary", image_size=(120, 160), batch_size=128, class_names=["controls", "patients"]
)

Found 708 files belonging to 2 classes.


In [4]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 120, 160, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [5]:
def normalize_images(image, label):
    return image / 255.0, label

dataset = dataset.map(normalize_images)

In [6]:
dataset

<_MapDataset element_spec=(TensorSpec(shape=(None, 120, 160, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [7]:
dataset_size = len(dataset)

# train 데이터셋 크기 계산
train_size = int(0.8 * dataset_size)

# 데이터셋을 나누기
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

# 나눠진 데이터셋을 확인
print("Train dataset:", len(train_dataset))
print("Test dataset:", len(test_dataset))

Train dataset: 4
Test dataset: 2


In [8]:
train_dataset

<_TakeDataset element_spec=(TensorSpec(shape=(None, 120, 160, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [9]:
test_dataset

<_SkipDataset element_spec=(TensorSpec(shape=(None, 120, 160, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

# CNN Modeling

In [10]:
from keras.models import Sequential
from keras.layers import Dropout, Activation, Dense
from keras.layers import Flatten, Convolution2D, MaxPooling2D
from keras.models import load_model
import tensorflow as tf
import cv2

batch_size = 20
num_classes = 1
model = Sequential()
model.add(Convolution2D(64, 3, 3, padding='same',activation='relu',
                        input_shape=(120,160,3)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(128, 3, 3, padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(0.25))

model.add(Convolution2D(128, 3, 3,padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = 'relu')) #relu
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation = 'sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='Adam',
              metrics=['accuracy'])
model.fit(train_dataset, batch_size=20, epochs=100)

#model.save('Gersang.h5')

Epoch 1/100
4/4 [==============================] - 80s 421ms/step - loss: 0.7041 - accuracy: 0.4902
Epoch 2/100
4/4 [==============================] - 11s 441ms/step - loss: 0.7070 - accuracy: 0.4609
Epoch 3/100
4/4 [==============================] - 10s 403ms/step - loss: 0.6975 - accuracy: 0.4941
Epoch 4/100
4/4 [==============================] - 9s 707ms/step - loss: 0.6936 - accuracy: 0.5547
Epoch 5/100
4/4 [==============================] - 9s 417ms/step - loss: 0.6966 - accuracy: 0.4844
Epoch 6/100
4/4 [==============================] - 10s 699ms/step - loss: 0.6914 - accuracy: 0.5215
Epoch 7/100
4/4 [==============================] - 8s 404ms/step - loss: 0.7033 - accuracy: 0.4609
Epoch 8/100
4/4 [==============================] - 11s 399ms/step - loss: 0.6974 - accuracy: 0.5039
Epoch 9/100
4/4 [==============================] - 9s 405ms/step - loss: 0.6927 - accuracy: 0.5156
Epoch 10/100
4/4 [==============================] - 10s 688ms/step - loss: 0.6949 - accuracy: 0.5195
Epo

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 40, 54, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 20, 27, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 20, 27, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 9, 128)         73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 4, 5, 128)         0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 4, 5, 128)         0

In [18]:
model.evaluate(test_dataset)

2/2 [==============================] - 7s 52ms/step - loss: 0.1375 - accuracy: 0.9694


[0.13754120469093323, 0.9693877696990967]

# Case Application

In [15]:
case_dataset = keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/의학영상처리/case_application_dataset", labels="inferred", label_mode="binary", image_size=(120, 160), batch_size=128, class_names=["controls", "patients"]
)

Found 32 files belonging to 2 classes.


In [16]:
case_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 120, 160, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [17]:
def normalize_images(image, label):
    return image / 255.0, label

case_dataset = case_dataset.map(normalize_images)

In [19]:
model.evaluate(case_dataset)

1/1 [==============================] - 5s 5s/step - loss: 0.3412 - accuracy: 0.8438


[0.341203510761261, 0.84375]